## キャンセルのトランザクションについて
- Quantity < 0: 10624 レコード
  - キャンセル: 9288 レコード
  - not キャンセル: 1336 レコード --> 棚卸し？

### 結論
- Quantity < 0 and not キャンセルは、棚卸し処理のために使われているように見える。

In [1]:
import time

from IPython.core.display import display
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
import numpy as np
import pandas as pd

In [2]:
data_dir = '../data/'
file = 'online_retail.xlsx'

t0 = time.time()
df_origin = pd.read_excel(data_dir + file)
print('Time = ', time.time() - t0)

Time =  49.1351420879364


In [3]:
# キャンセルを示すレコードの数
df_origin['InvoiceNo'].str.contains('c', case=False).sum()

9288

In [7]:
# キャンセルのレコード
df_origin[df_origin['InvoiceNo'].str.contains('c', case=False) == True].head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527.0,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311.0,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548.0,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom


In [4]:
df_cancel = df_origin[df_origin['InvoiceNo'].str.contains('c', case=False) == True]

In [5]:
df_cancel['Sales'] = df_cancel['Quantity'] * df_cancel['UnitPrice']

/Users/fujiki/anaconda3/envs/DataScience/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [6]:
df_cancel.head(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Sales
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527.0,United Kingdom,-27.50
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311.0,United Kingdom,-4.65
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548.0,United Kingdom,-19.80


In [8]:
df_cancel['Sales'].sum()

-896812.48999999999

In [7]:
# Discount のレコード数
df_origin[df_origin['StockCode'] == 'D'].shape

(77, 8)

In [10]:
df_sales['Sales'].sum()

10666684.544

In [12]:
df_cancel['InvoiceNo'].nunique()

3836

In [33]:
# Quantity < 0 のレコードの数
# NOTE: Quantity < 0 だが キャンセルではないレコードがある？
df_cancel = \
df_origin[df_origin['Quantity'] < 0]
print(df_cancel.shape)

(10624, 8)


In [11]:
# Quantity < 0 and キャンセルのレコード数
# NOTE:
# Quantity < 0 のレコードの集合の中に、キャンセルのレコードの集合が包含されているらしい。

df_minus_quantity = df_origin[df_origin['Quantity'] < 0]
df_minus_quantity[df_minus_quantity['InvoiceNo'].str.contains('c', case=False) == True].shape

(9288, 8)

In [30]:
# Quantity < 0 だが not キャンセルのレコード

df_minus_quantity_and_not_cancel = \
df_minus_quantity[~(df_minus_quantity['InvoiceNo'].str.contains('c', case=False) == True)]
print(df_minus_quantity_and_not_cancel.shape)

(1336, 8)


In [37]:
# Quantity < 0 だが not キャンセルのレコード
# NOTE:
# すべて UnitPrice == 0.0 らしい。

df_minus_quantity_and_not_cancel[df_minus_quantity_and_not_cancel['UnitPrice'] == 0.0].shape

(1336, 8)

In [39]:
# Quantity < 0 だが not キャンセルのレコード
# NOTE:
# そして、すべて Country は UK

df_minus_quantity_and_not_cancel['Country'].unique()

array(['United Kingdom'], dtype=object)

In [41]:
# Quantity < 0 だが not キャンセルのレコード
# NOTE:
# そして、CustomerID はすべて NaN

df_minus_quantity_and_not_cancel['CustomerID'].isnull().sum()

1336

In [42]:
df_minus_quantity_and_not_cancel

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
2406,536589,21777,NaN,-10,2010-12-01 16:50:00,0.0,NaN,United Kingdom
4347,536764,84952C,NaN,-38,2010-12-02 14:42:00,0.0,NaN,United Kingdom
7188,536996,22712,NaN,-20,2010-12-03 15:30:00,0.0,NaN,United Kingdom
7189,536997,22028,NaN,-20,2010-12-03 15:30:00,0.0,NaN,United Kingdom
7190,536998,85067,NaN,-6,2010-12-03 15:30:00,0.0,NaN,United Kingdom
7192,537000,21414,NaN,-22,2010-12-03 15:32:00,0.0,NaN,United Kingdom
7193,537001,21653,NaN,-6,2010-12-03 15:33:00,0.0,NaN,United Kingdom
7195,537003,85126,NaN,-2,2010-12-03 15:33:00,0.0,NaN,United Kingdom
7196,537004,21814,NaN,-30,2010-12-03 15:34:00,0.0,NaN,United Kingdom
7197,537005,21692,NaN,-70,2010-12-03 15:35:00,0.0,NaN,United Kingdom
